In [3]:
import pandas as pd # library for data analysis
import requests # library to handle requests
from bs4 import BeautifulSoup # library to parse HTML documents

import plotly.express as px

from sklearn.metrics import confusion_matrix
import numpy as np

/Users/jackohagan/miniforge3/envs/tensorflow/lib/python3.9/site-packages/scipy/__init__.py:146: UserWarning: A NumPy version >=1.16.5 and <1.23.0 is required for this version of SciPy (detected version 1.23.0
  warnings.warn(f"A NumPy version >={np_minversion} and <{np_maxversion}"


In [11]:
candidates = pd.read_csv('candidates.csv')
couples = pd.read_csv('couples.csv')
ratings = pd.read_csv('ratings.csv')


In [12]:
candidates.head()

,Unnamed: 0,Islander,Age,Home town,Entered,Status,Ref,series,Hometown,Ref.
0,0,Jess Hayes,22,Oxford,Day 1,Winner (Day 41),[12],1,NaN,NaN
1,1,Max Morley,22,Huddersfield,Day 14,Winner (Day 41),[13],1,NaN,NaN
2,2,Hannah Elizabeth,25,Liverpool,Day 1,Runner-up (Day 41),[14],1,NaN,NaN
3,3,Jon Clark,25,Essex,Day 1,Runner-up (Day 41),[15],1,NaN,NaN
4,4,Josh Ritchie,21,Bolton,Day 1,Third place (Day 41),[16],1,NaN,NaN


In [13]:
couples.head()

,Unnamed: 0,Unnamed: 0.1,Day 1,Day 6,Day 11,Day 33,Final,series,Day 13,Day 20,...,Day 18,Day 26,Unnamed: 8,Day 19,Day 25,Day 45,Day 12,Day 23,Day 31,Day 38
0,0,NaN,NaN,NaN,NaN,NaN,NaN,1,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,1,Jess,Jordan,Josh,Omar,Max,Winner (Day 41),1,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,2,Max,Not in Villa,Not in Villa,Not in Villa,Jess,Winner (Day 41),1,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,3,Hannah,Jon,Jon,Jon,Jon,Runner-up (Day 41),1,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,4,Jon,Hannah,Hannah,Hannah,Hannah,Runner-up (Day 41),1,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [14]:
##tidy up entry and status column

##replace non text
candidates['entered_day'] = candidates['Entered'].str.replace('[^0-9]', '').astype(int)

candidates['exit_day'] = candidates['Status'].str.replace('[^0-9]', '').astype(int)

##split out the winner/runner up etc 
candidates['status'] = candidates['Status'].str.replace('Day','').str.replace('[^a-zA-Z -]', '').str.strip()

##fill different hometown column
candidates['Hometown'] = candidates['Hometown'].fillna(candidates['Home town'])

/var/folders/_9/dygt4j8d1s1197cm8ymg58cm0000gn/T/ipykernel_2945/1587096268.py:4: FutureWarning: The default value of regex will change from True to False in a future version.
  candidates['entered_day'] = candidates['Entered'].str.replace('[^0-9]', '').astype(int)
/var/folders/_9/dygt4j8d1s1197cm8ymg58cm0000gn/T/ipykernel_2945/1587096268.py:6: FutureWarning: The default value of regex will change from True to False in a future version.
  candidates['exit_day'] = candidates['Status'].str.replace('[^0-9]', '').astype(int)
/var/folders/_9/dygt4j8d1s1197cm8ymg58cm0000gn/T/ipykernel_2945/1587096268.py:9: FutureWarning: The default value of regex will change from True to False in a future version.
  candidates['status'] = candidates['Status'].str.replace('Day','').str.replace('[^a-zA-Z -]', '').str.strip()


In [15]:
##drop uncessary columns
candidates.drop(columns=['Unnamed: 0', 'Entered','Home town','Status','Ref','Ref.'],inplace=True)

Metrics

In [16]:

##create winner flag

candidates['won'] = (candidates['status'] == 'Winner')


candidates['og'] = (candidates['entered_day'] == 1)

#create time in show
candidates['time_in_show'] = candidates['exit_day'] - candidates['entered_day']

candidates.head()

candidates.to_csv('candidatestidied.csv')

In [17]:
##tidy couples now
##rename columns and convert to rank

couples.head(10)
couples.rename({'Unnamed: 0.1':'Candidate'}, axis= 1,inplace=True)
couples.drop(['Unnamed: 0', 'Unnamed: 10',
       'Unnamed: 11', 'Unnamed: 8', 'Unnamed: 9'], axis=1, inplace=True)

couples.sort_index(axis=1, inplace=True)


In [18]:
##convert columns to rows to get one row per person per day

couples_melt = couples.melt(id_vars=["Candidate", "series"], 
        var_name="Recoupling", 
        value_name="Couple")



In [19]:
##remove nulls
couples_melt = couples_melt[couples_melt['Couple'].notna()]

In [20]:
## does output looks sensible
couples_melt[couples_melt['Candidate']== 'Jess'].sort_values(['series','Recoupling'],ascending =True)

,Candidate,series,Recoupling,Couple
1,Jess,1,Day 1,Jordan
569,Jess,1,Day 11,Omar
5113,Jess,1,Day 33,Max
8237,Jess,1,Day 6,Josh
8521,Jess,1,Final,Winner (Day 41)
178,Jess,6,Day 1,—
1314,Jess,6,Day 13,Luke M
1882,Jess,6,Day 18,Luke M
3870,Jess,6,Day 26,Ched
5290,Jess,6,Day 33,Ched


Clean table, remove any day value on recoupling and other unecessary valuea

In [20]:
##clean up day and then use it as a ranking column
couples_melt['recoupling_day'] = couples_melt['Recoupling'].str.replace('[^0-9]', '')

##remove booted date as that's in candidates
couples_melt = couples_melt[couples_melt.recoupling_day != '']

##remove rows where peopel are dumped,walk, removed or not yet in villa
couples_melt = couples_melt[couples_melt['Couple'].str.contains('Dumped') == False]
couples_melt = couples_melt[couples_melt['Couple'].str.contains('Walked') == False]
couples_melt = couples_melt[couples_melt['Couple'].str.contains('Not in Villa') == False]
couples_melt = couples_melt[couples_melt['Couple'].str.contains('Removed') == False]


##exclude where people were in, but not part of recoupling.
couples_melt = couples_melt[couples_melt['Couple'].str.contains('—') == False]

##
##rename left single 
#couples_melt['Couple'] = couples_melt['Couple'].str.replace('—', 'Left Single')


##exclude wikipedia notes as a column
couples_melt = couples_melt[couples_melt['Candidate'].str.contains('Notes') == False]

## convert to int
couples_melt['recoupling_day'] = couples_melt['recoupling_day'].astype(int)

/var/folders/_9/dygt4j8d1s1197cm8ymg58cm0000gn/T/ipykernel_2036/2884342711.py:2: FutureWarning: The default value of regex will change from True to False in a future version.
  couples_melt['recoupling_day'] = couples_melt['Recoupling'].str.replace('[^0-9]', '')


In [22]:
##clean up and write to csv, for network graph
couples_clean = couples_melt[['Candidate','Couple', 'series']]

##add series to name
#couples_clean['Candidate'] = couples_clean['Candidate'] +' - '+ couples_clean['series'].astype(str)

#couples_clean['Couple'] = couples_clean['Couple'] +' - '+ couples_clean['series'].astype(str)

##clean up and write to csv, for network graph
couples_clean = couples_clean[['Candidate','Couple','series']]

couples_clean.to_csv('couples_network.csv')

In [23]:
##now rank based on recoupling_day
couples_melt['rank'] = couples_melt.groupby(['Candidate','series'])['recoupling_day'].rank(method="first", ascending=True)


In [24]:
##rank by couples
couples_melt = couples_melt.sort_values(['series','recoupling_day','Candidate','rank'], ascending=True)

In [25]:
##looks sensible
couples_melt[couples_melt['Candidate']== 'Olivia'].sort_values(['series','recoupling_day'],ascending =True)

,Candidate,series,Recoupling,Couple,recoupling_day,rank
30,Olivia,2,Day 1,Daniel,1,1.0
8266,Olivia,2,Day 6,Rykard,6,2.0
1166,Olivia,2,Day 13,Adam M,13,3.0
2302,Olivia,2,Day 20,Alex,20,4.0
4858,Olivia,2,Day 32,Alex,32,5.0
5710,Olivia,2,Day 37,Alex,37,6.0
6562,Olivia,2,Day 40,Alex,40,7.0
61,Olivia,3,Day 1,Marcel,1,1.0
8297,Olivia,3,Day 6,Sam,6,2.0
1481,Olivia,3,Day 14,Mike,14,3.0


In [26]:
##create lag to see how long people stick before moving on
couples_melt = couples_melt.sort_values(by=['Candidate','series','rank'],ascending=False)
#applying leads and lags by shifting dataframe column up and down respectively
couples_melt.groupby(['Candidate','series'])['Couple'].shift(-1)

couples_melt

,Candidate,series,Recoupling,Couple,recoupling_day,rank
5122,Zoe,1,Day 33,Jordan,33,4.0
578,Zoe,1,Day 11,Chris W,11,3.0
8246,Zoe,1,Day 6,Jordan,6,2.0
10,Zoe,1,Day 1,Luis,1,1.0
2394,Zara,4,Day 20,Adam,20,1.0
...,...,...,...,...,...,...
2221,Aaron F,7,Day 19,Kaz,19,4.0
1653,Aaron F,7,Day 14,Kaz,14,3.0
8185,Aaron F,7,Day 5,Sharon,5,2.0
233,Aaron F,7,Day 1,Shannon,1,1.0


In [27]:
##candidates list agg
couplesgroup = couples_melt.groupby(['Candidate','series']).agg({'Couple':lambda x: list(x)})

In [28]:
##first and last partner
couplesgroup['first_partner'] = [l[0] for l in couplesgroup['Couple']]

##first and last partner
couplesgroup['last_partner'] = [l[-1] for l in couplesgroup['Couple']]


In [29]:

##number of unique partners per person 
couples_unique = couples_melt.groupby(['Candidate','series'])['Couple'].agg(['count','nunique']).reset_index()

couples_unique.rename({'count':'total_recouples','nunique':'unique_partners'},axis=1, inplace=True)

In [30]:
##merge both dataframes on candidate and serise

final_couples = couplesgroup.merge(couples_unique, on=['Candidate','series'])

final_couples

,Candidate,series,Couple,first_partner,last_partner,total_recouples,unique_partners
0,AJ,7,[Hugo],Hugo,Hugo,1,1
1,Aaron F,7,"[Lucinda, Kaz, Kaz, Sharon, Shannon]",Lucinda,Shannon,5,4
2,Aaron S,7,[Mary],Mary,Mary,1,1
3,Abigail,7,"[Dale, Toby]",Dale,Toby,2,2
4,Adam,4,"[Darylle, Zara, Rosie, Rosie]",Darylle,Rosie,4,3
...,...,...,...,...,...,...,...
202,Wes,4,"[Megan, Megan, Laura A, Laura A, Laura A, Laur...",Megan,Laura A,6,2
203,Yewande,5,"[Danny, Michael, Michael]",Danny,Michael,3,2
204,Zara,2,"[Adam M, Daniel, Daniel, Scott]",Adam M,Scott,4,3
205,Zara,4,[Adam],Adam,Adam,1,1


In [31]:
##sort by number of partners

final_couples.sort_values('unique_partners', ascending = False).head(10)

,Candidate,series,Couple,first_partner,last_partner,total_recouples,unique_partners
54,Danica,8,"[Jamie, Billy, Josh, Jay, Jacques, Luca]",Jamie,Luca,6,6
174,Sam,3,"[Georgia, Chloë, Montana, Olivia, Camilla]",Georgia,Camilla,5,5
150,Montana,3,"[Alex, Alex, Alex, Dom, Simon, Sam, Marcel, Dom]",Alex,Dom,8,5
21,Anton,5,"[Belle, Belle, Belle, Lucie, Elma, Amber, Amy]",Belle,Amy,7,5
14,Amber,5,"[Greg, Ovie, Michael, Michael, Anton, Callum]",Greg,Callum,6,5
7,Adam M,2,"[Katie, Tina, Liana, Zara, Olivia]",Katie,Olivia,5,5
99,Jess,1,"[Max, Omar, Josh, Jordan]",Max,Jordan,4,4
152,Nas,6,"[Eva, Demi, Siânnise, Jess, Siânnise]",Eva,Siânnise,5,4
1,Aaron F,7,"[Lucinda, Kaz, Kaz, Sharon, Shannon]",Lucinda,Shannon,5,4
30,Camilla,3,"[Jamie, Jamie, Craig, Jonny, Jonny, Jonny, Jon...",Jamie,Sam,8,4


In [32]:
##write to csv

final_couples.to_csv('final_couples.csv')